In [4]:
from sklearn.feature_selection import mutual_info_regression, f_regression
from sklearn.feature_selection import SelectPercentile
from sklearn.linear_model import RandomizedLasso, Ridge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [5]:
from subprocess import check_output


In [6]:
#df = pd.read_csv(r'/home/pirwin/Git/IPy_Notebooks/SQL/CNTYVARS.csv')
df = pd.read_csv(r'C:\Users\pairwin\Documents\GitHub\IPy_Notebooks\SQL\CNTYVARS.csv')
df['HSGRAD_PRCNT'] = df['HSGRAD_CY']/df['TOTPOP_CY']

In [7]:
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
print(dtype_df[dtype_df['Column Type'] == 'object'])

         Count Column Type
783   TSEGCODE      object
784   TSEGNAME      object
854  TLIFECODE      object
855  TLIFENAME      object


In [8]:
dp = ['OBJECTID','Customers', 'UNITS', 'SALES', 'LOG_UNITS', 'TSEGCODE','TSEGNAME','TLIFECODE','TLIFENAME']

X = df.drop(dp, axis=1)

#X2 = transformer.fit_transform(X, y)

#X2.head()

In [9]:
y = df[['LOG_UNITS']]
y.shape

(3109, 1)

In [10]:
selector = SelectPercentile(f_regression, percentile=20)
columns = X.columns
selector.fit_transform(X, np.array(y))
labels = [columns[x] for x in selector.get_support(indices=True) if x]
XNew = pd.DataFrame(selector.fit_transform(X, np.array(y)), columns = labels)
columns = XNew.columns


lst = list()
for n, s in zip(columns, selector.scores_):
    lst.append((s, n))

scores = pd.DataFrame(lst, columns=['Score','Name'])
    


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:279: RuntimeWarning: invalid value encountered in true_divide
  corr /= row_norms(X.T)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [11]:
inds = list()
for val in columns:
    #print(val)
    inds.append((val, XNew.columns.get_loc(val)))
    
indices = pd.DataFrame(inds, columns=['Name','XNew_Index'])
    


In [12]:
scores.sort_values(by='Score',inplace=True, ascending=True)
scores.reset_index(inplace=True)
#scores.to_csv(r'C:\Users\pairwin\Documents\GitHub\IPy_Notebooks\SQL\varscores.csv')
#scores.to_csv(r'/home/pirwin/Git/IPy_Notebooks/SQL/varscores.csv')

In [13]:
scores.drop('index', inplace=True, axis = 1)

df = pd.merge(scores, indices, how='inner',left_on='Name',right_on='Name')



In [14]:
def calculate_vif(X, thresh=7.5):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            # Loop repeatedly until we find that all columns within our dataset
            # have a VIF value we're happy with.
            variables = X.columns
            dropped=False
            vif = []
            new_vif = 0
            for var in variables:
                new_vif = variance_inflation_factor(X.as_matrix(), X.columns.get_loc(var))
                vif.append(new_vif)
                if np.isinf(new_vif):
                    break
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)                
                print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                #df = df[df['Name'] != str(X.columns[maxloc])]
                dropped=True
        return X

In [13]:
finaldf = calculate_vif(XNew)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Dropping POP60_CY with vif=inf
Dropping POP65_CY with vif=inf
Dropping POP70_CY with vif=inf
Dropping POP75_CY with vif=inf
Dropping POP80_CY with vif=inf
Dropping MALE60_CY with vif=inf
Dropping MALE65_CY with vif=inf
Dropping MALE70_CY with vif=inf
Dropping MALE75_CY with vif=inf
Dropping MALE80_CY with vif=inf
Dropping FEM60_CY with vif=inf
Dropping FEM65_CY with vif=inf
Dropping FEM70_CY with vif=inf
Dropping FEM75_CY with vif=inf
Dropping FEM80_CY with vif=inf
Dropping AGE58_CY with vif=inf
Dropping AGE59_CY with vif=inf
Dropping AGE60_CY with vif=inf
Dropping AGE61_CY with vif=inf
Dropping AGE62_CY with vif=inf
Dropping AGE63_CY with vif=inf
Dropping AGE64_CY with vif=inf
Dropping AGE65_CY with vif=inf
Dropping AGE66_CY with vif=inf
Dropping AGE67_CY with vif=inf
Dropping AGE68_CY with vif=inf
Dropping AGE69_CY with vif=inf
Dropping AGE70_CY with vif=inf
Dropping AGE71_CY with vif=inf
Dropping AGE72_CY with vif=inf
Dropping AGE73_CY with vif=inf
Dropping AGE74_CY with vif=inf
Dro

Dropping A65DI25_CY with vif=94.55575751181497
Dropping NHSPWHT_CY with vif=87.3042463056741
Dropping S10_SALES with vif=83.42918105391244
Dropping S07_BUS with vif=70.55125798000711
Dropping ASSCDEG_CY with vif=68.02361151528055
Dropping N18_SALES with vif=67.11072341061796
Dropping VAL150K_CY with vif=63.04939362768436
Dropping N09_BUS with vif=61.118862670093456
Dropping A55DI25_CY with vif=58.48534938695471
Dropping A25NW150CY with vif=51.27744558263795
Dropping S03_EMP with vif=45.53606446417483
Dropping A75DI15_CY with vif=41.415385931598365
Dropping N13_EMP with vif=26.68134444977258
Dropping GED_CY with vif=25.029237495197147
Dropping AGGNWA55CY with vif=23.02159747653444
Dropping A15NW35_CY with vif=21.605432093256937
Dropping N12_EMP with vif=21.586376596797745
Dropping N09_EMP with vif=17.86754110938265
Dropping S31_BUS with vif=15.281064840908478
Dropping N33_EMP with vif=15.1587870543486
Dropping S02_BUS with vif=14.561249018314431
Dropping INDUTIL_CY with vif=14.025132649

In [14]:
finaldf.tail()

,INDPUBL_CY,VAL0_CY,VAL100K_CY,THHGRPL4,THHGRPL5,THHGRPL6,THHGRPU5,THHGRPU6,TADULT21,TADULT38,S24_BUS,S31_EMP,N04_BUS
3104,191.0,346.0,419.0,0.0,0.0,1944.0,992.0,1871.0,0.0,0.0,7.0,282.0,1.0
3105,2454.0,1081.0,1807.0,3647.0,3310.0,8228.0,4306.0,7802.0,9487.0,737.0,32.0,1630.0,7.0
3106,248.0,137.0,234.0,0.0,0.0,2093.0,0.0,2598.0,0.0,0.0,23.0,293.0,1.0
3107,423.0,430.0,788.0,855.0,4080.0,2032.0,1348.0,3336.0,0.0,1555.0,57.0,722.0,9.0
3108,778.0,621.0,1855.0,0.0,0.0,7934.0,3952.0,7939.0,0.0,0.0,33.0,1566.0,5.0


In [15]:
rlasso = RandomizedLasso(alpha=0.001)
rlasso.fit(X, np.ravel(y))

RandomizedLasso(alpha=0.001, eps=2.2204460492503131e-16, fit_intercept=True,
        max_iter=500, memory=Memory(cachedir=None), n_jobs=1,
        n_resampling=200, normalize=True, pre_dispatch='3*n_jobs',
        precompute='auto', random_state=None, sample_fraction=0.75,
        scaling=0.5, selection_threshold=0.25, verbose=False)

In [16]:
print("Features sorted by their score:")
lassoscored = pd.DataFrame(sorted(zip(map(lambda x: round(x, 4), rlasso.scores_), 
                 X.columns), reverse=True), columns=['Score','Feature'])
lassoscored = lassoscored[lassoscored['Score'] > 0]
lassolist = lassoscored['Feature'].as_matrix().tolist()
print(lassoscored)

Features sorted by their score:
     Score     Feature
0    0.870  MEDNWA15CY
1    0.750       THH47
2    0.740    THHGRPU6
3    0.740   MEDVAL_CY
4    0.730    LANDAREA
5    0.710   MEDHHR_CY
6    0.620       THH24
7    0.570  UNEMPRT_CY
8    0.545    THHGRPU5
9    0.535    TADULT58
10   0.515  AVGNWA65CY
11   0.495     N40_BUS
12   0.490   THHGRPL10
13   0.445  AVGNWA55CY
14   0.445  AVGNWA45CY
15   0.440  AVGNWA35CY
16   0.435       THH48
17   0.375    TADULT18
18   0.360  MEDMAGE_CY
19   0.355       THH22
20   0.340  POPGRW10CY
21   0.315       THH25
22   0.300       THH58
23   0.300       THH13
24   0.295  AVGDIA15CY
25   0.285    THHGRPL6
26   0.265    TADULT47
27   0.260    TADULT24
28   0.250       THH50
29   0.250       THH15
..     ...         ...
240  0.005  AVGIA65UCY
241  0.005  AVGIA55UCY
242  0.005  AVGDIA25CY
243  0.005  ASNMBASECY
244  0.005  AIMBASE_CY
245  0.005  AGGNWA55CY
246  0.005  AGGNWA35CY
247  0.005     AGE6_CY
248  0.005  A75NW35_CY
249  0.005  A75DI75_CY
25

In [17]:
XNew2 = X[lassolist]

In [18]:
XNew2.head()

,MEDNWA15CY,THH47,THHGRPU6,MEDVAL_CY,LANDAREA,MEDHHR_CY,THH24,UNEMPRT_CY,THHGRPU5,TADULT58,...,A45DI200CY,A35I150_CY,A35I100_CY,A35DI35_CY,A25NW15_CY,A25NW100CY,A25DI75_CY,A25DI25_CY,A15I15_CY,A15I0_CY
0,10426,0,856,590884,739.0174,50.3,0,6.4,5417,2288,...,9964,12716,23008,14892,13441,3260,14012,8985,2887,5438
1,13472,0,7783,284436,594.5831,60.4,0,8.0,2224,872,...,61,104,295,307,215,82,160,136,26,44
2,8933,1609,9415,259717,1636.4644,53.5,830,8.7,18894,2415,...,367,590,1585,2197,1758,419,1198,1848,1377,2889
3,15574,0,10535,256835,1020.0123,59.9,0,5.3,4341,1442,...,101,108,336,332,246,91,213,176,38,44
4,11583,0,1744,214455,1150.7310,52.1,594,7.0,2400,539,...,40,34,193,239,168,62,124,158,43,51


In [19]:
lassofinal = calculate_vif(XNew2, thresh=5)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Dropping THHGRPU6 with vif=inf
Dropping THH47 with vif=inf
Dropping THH24 with vif=inf
Dropping S24_BUS with vif=inf
Dropping AIFBASE_CY with vif=inf
Dropping S11_EMP with vif=inf
Dropping HISPBLK_CY with vif=inf
Dropping BLACK_CY with vif=inf
Dropping AMERIND_CY with vif=inf
Dropping NHSPAI_CY with vif=inf
Dropping OWNER_CY with vif=514515344.0505184
Dropping N40_EMP with vif=287699.2282302107
Dropping N40_BUS with vif=156938.00902398853
Dropping BAGEBASECY with vif=86578.013490416
Dropping THHGRPL8 with vif=29325.340608440263
Dropping TADULT61 with vif=25687.798221554265
Dropping THHGRPU4 with vif=22005.93946469276
Dropping BLKMBASECY with vif=14264.141611103047
Dropping THH36 with vif=12114.12556680235
Dropping VALBASE_CY with vif=11927.103299400897
Dropping A55NW250CY with vif=10751.9281605092
Dropping THHGRPU5 with vif=7252.339164521409
Dropping DI35_CY with vif=7063.22771256425
Dropping THHGRPL6 with vif=7024.716372271954
Dropping S32_EMP with vif=3538.6338014068615
Dropping TADU

Dropping N21_EMP with vif=11.341218252782417
Dropping THH61 with vif=10.994242110962688
Dropping THH04 with vif=10.748664820248667
Dropping INDMGMT_CY with vif=9.995116952383862
Dropping N32_SALES with vif=9.848193308978864
Dropping TADULT36 with vif=9.751268018937585
Dropping N02_EMP with vif=9.687651043345037
Dropping THH52 with vif=9.279292463243765
Dropping MEDVAL_CY with vif=8.988501452949984
Dropping TSEGNUM with vif=8.731347990090482
Dropping TADULT41 with vif=8.426938413550841
Dropping N04_EMP with vif=8.252891966044503
Dropping S24_EMP with vif=8.080173571083707
Dropping S31_EMP with vif=7.789715871477655
Dropping THH06 with vif=7.645311106771169
Dropping S21_EMP with vif=7.574919372999211
Dropping AVGNWA15CY with vif=7.2226854580422195
Dropping N15_EMP with vif=7.178820008900161
Dropping TADULT27 with vif=7.030733862369069
Dropping TADULT65 with vif=6.9837549741344
Dropping THH37 with vif=6.823756229734211
Dropping N02_BUS with vif=6.740389810560609
Dropping THH45 with vif=6.

In [20]:
lassofinal.head()

,LANDAREA,TADULT58,THH48,TADULT18,THH22,THH13,TADULT47,TADULT24,THH50,TADULT43,...,N34_EMP,N30_SALES,N30_EMP,N20_SALES,N17_EMP,N03_SALES,N02_SALES,MEDNWA25CY,IAGEBASECY,DIVINDX_CY
0,739.0174,2288,0,0,0,4102,0,0,0,7119,...,15377,58717,191,817549,4268,30207,75637,15020,9670,82.9
1,594.5831,872,0,0,0,0,0,0,0,3125,...,1903,0,0,6059,87,12000,17995,17737,704,43.1
2,1636.4644,2415,334,1611,0,0,3596,1773,908,29493,...,2105,4004,157,57306,768,34393,127390,11450,4730,53.3
3,1020.0123,1442,0,0,0,0,0,0,0,6452,...,365,0,0,4693,123,4014,10975,16458,707,40.1
4,1150.7310,539,0,0,0,0,0,1206,0,564,...,553,0,0,3320,18,3840,160880,13206,386,80.2


In [24]:
import statsmodels.api as sts


In [25]:
XNew3 = sts.add_constant(lassofinal)
mod = sts.OLS(y, XNew3)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              LOG_UNITS   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     77.97
Date:                Thu, 07 Sep 2017   Prob (F-statistic):               0.00
Time:                        16:30:35   Log-Likelihood:                -4079.5
No. Observations:                3109   AIC:                             8275.
Df Residuals:                    3051   BIC:                             8625.
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.5518      0.060     91.837      0.0

In [29]:
var = ['const','THH54','TADULT40','TADULT42','N30_EMP','THH65','N41_EMP','TADULT24','TADULT66','N30_SALES',
       'THH31','THH59','TADULT43','LANDAREA','TADULT58','THH48','TADULT18','THH22','THH13','TADULT47',
       'THH50','N04_SALES','TADULT25','TADULT15','TADULT26','THH17','THH46','THH21','THH23','TADULT09','FAMGRW10CY',
       'THH07','PIMBASE_CY','MEDNWA25CY','DIVINDX_CY']
XNew4 = XNew3[var]

In [30]:
mod = sts.OLS(y, XNew4)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              LOG_UNITS   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.579
Method:                 Least Squares   F-statistic:                     126.9
Date:                Thu, 07 Sep 2017   Prob (F-statistic):               0.00
Time:                        16:33:51   Log-Likelihood:                -4113.4
No. Observations:                3109   AIC:                             8297.
Df Residuals:                    3074   BIC:                             8508.
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.5848      0.059     94.176      0.0

In [34]:
#%matplotlib inline
cor = XNew4.corr()
print(cor)


            const     THH54  TADULT40  TADULT42   N30_EMP     THH65   N41_EMP  \
const         NaN       NaN       NaN       NaN       NaN       NaN       NaN   
THH54         NaN  1.000000  0.265463  0.325358  0.306776  0.184506  0.393301   
TADULT40      NaN  0.265463  1.000000  0.703985  0.054904  0.053342  0.127520   
TADULT42      NaN  0.325358  0.703985  1.000000  0.062395  0.050839  0.183553   
N30_EMP       NaN  0.306776  0.054904  0.062395  1.000000  0.043965  0.609306   
THH65         NaN  0.184506  0.053342  0.050839  0.043965  1.000000  0.188602   
N41_EMP       NaN  0.393301  0.127520  0.183553  0.609306  0.188602  1.000000   
TADULT24      NaN -0.090562 -0.066780 -0.046831 -0.052578 -0.057483 -0.099456   
TADULT66      NaN  0.499709  0.092877  0.149664  0.289438  0.124977  0.361373   
N30_SALES     NaN  0.152164  0.048958  0.049583  0.723412  0.027288  0.413118   
THH31         NaN  0.054260  0.089569  0.066377  0.011602  0.074384  0.040713   
THH59         NaN  0.364161 